### Собираем ссылки на все файлы
https://brainout.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B

In [7]:
import requests
from bs4 import BeautifulSoup

url = 'https://brainout.fandom.com/ru/wiki/Служебная:Все_страницы?from=&to=&namespace=6'

In [ ]:
def file_in_string(string):
    return str(string).startswith('Файл:') or str(string).startswith('File:')

def next_in_string(string):
    return str(string).startswith('Следующая страница') or str(string).startswith('Next page')

start_url = 'https://brainout.fandom.com/ru/wiki/Служебная:Все_страницы?from=&to=&namespace=6'
response = requests.get(start_url)
soup = BeautifulSoup(response.text, 'html.parser')

next_page = soup.find(title="Служебная:Все страницы", string=next_in_string)
url_arr=[]

while next_page:
    print('Startin url append')
    for file in soup.find_all('a', string=file_in_string):
        url_arr.append('https://brainout.fandom.com' + file.get('href'))
    
    next_page = soup.find(title="Служебная:Все страницы", string=next_in_string)
    print(next_page)
    if next_page:
        next_url = next_page.get('href')
    else:
        print('goin to break')
        break
    print('Goin for next page', next_url)
    response = requests.get('https://brainout.fandom.com' + next_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #print(soup)

### Сохраняем результаты

In [81]:
import json
with open('url_arr.json', 'w') as f:
    json.dump(url_arr, f)

### Открываем результаты

In [1]:
import json
with open('url_arr.json', 'r', encoding="utf-8") as f:
    url_arr = json.load(f)

In [4]:
url_arr[0]

'https://brainout.fandom.com/ru/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:%22%D0%A1%D0%B2%D0%BE%D0%B1%D0%BE%D0%B4%D0%BD%D0%B0%D1%8F_%D0%B8%D0%B3%D1%80%D0%B0%22_%D0%B2_%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D0%B0%D1%85_%D0%B8_%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%B0%D1%85.png'

### Логинимся на страницу фандома
https://www.fandom.com/signin?uselang=ru&redirect=https%3A%2F%2Fbrainout.fandom.com%2Fru%2Ff

In [10]:
start_url = 'https://www.fandom.com/signin?uselang=ru&redirect=https%3A%2F%2Fbrainout.fandom.com%2Fru%2Ff'

In [48]:
%timeit
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def login_through(driver, start_url):
    driver.get(start_url)
    element = driver.find_element_by_id("loginUsername")
    element.send_keys("foxinol")
    element = driver.find_element_by_id("loginPassword")
    element.send_keys("aboba")
    driver.find_element_by_id("loginSubmit").click()
    

options = webdriver.ChromeOptions()
options.page_load_strategy = 'eager'
options.add_argument('--ignore-ssl-errors')
options.add_argument('--ignore-certificate-errors-spki-list')
options.add_argument('--ignore-ssl-errors')

driver = webdriver.Chrome(options=options)
login_through(driver, start_url)

### В течении логин сессии начинаем собирать ссылки на static страницы

In [ ]:
import json
def full_file(string):
    return str(string).startswith('Original file') or str(string).startswith('Оригинальный файл')

static_links = []

for count, url in enumerate(url_arr):
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    fin_file = soup.find(class_='internal')
    static_links.append(fin_file.get('href'))
    print(count, 'success')

with open('static_links.json', 'w') as f:
    json.dump(static_links, f) 

driver.close()

### Скачиваем контент и работаем с названиями файлов

In [72]:
import json
with open('url_arr_changed.json', 'r', encoding="utf-8") as f:
    url_arr_changed = json.load(f)
with open('static_links_changed.json', 'r', encoding="utf-8") as f:
    static_links_changed = json.load(f)

In [ ]:
def ultimate_string_check(file_name):
    file_name = file_name.replace('-', '').replace('.', '').replace('(', '').replace(')', '').replace('_', '')[:-3]
    if file_name.isalpha():
        return "alpha"
    elif file_name.isnumeric():
        return "number"
    elif file_name.isalnum():
        return "alnum"
    else:
        return "trash"

from urllib.parse import unquote
not_worked_links = []
for i in range(len(static_links)):
    file_name = unquote(url_arr_changed[i].split('/')[-1][22:])
    try:
        urllib.request.urlretrieve(static_links[i], "images/" + ultimate_string_check(file_name) + "/" + file_name)
    except OSError:
        not_worked_links.append(static_links[i])

### Сохраняем необработанные ссылки

In [105]:
with open('not_worked_links.json', 'w') as f:
    json.dump(not_worked_links, f) 